In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# In-depth tutorial
This tutorial/example aims to go a bit more in-depth into the code-base compared to the "Hello World" example, and discuss the major classes and functions, and how they relate to one another.

## Muon generation
The `tomopt.muon` contains functions and classes to deal with the generation and handling of muons.

`tomopt.muon.generation.generate_batch` will generate N muons on demand via random sampling. These are stored as an (N,5) tensor, with columns corresponding to (x,y,momentum,$\theta_x$,$\theta_y$), where $\theta$ is the angle between the z-axis and the trajectory of the muon in x & y. Currently x & y are uniform in $[0,1)$, momentum is fixed, and the $\theta$ is not properly sampled.

In [11]:
from tomopt.muon import generate_batch

In [12]:
generate_batch(10)

tensor([[ 6.2819e-01,  5.4702e-02,  5.0000e+00,  1.7183e-03, -6.5004e-02],
        [ 3.0021e-01,  1.1466e-01,  5.0000e+00, -4.1258e-02, -7.6060e-02],
        [ 2.6147e-01,  4.3348e-01,  5.0000e+00,  8.1472e-02,  5.5091e-02],
        [ 2.4232e-01,  6.2119e-01,  5.0000e+00, -2.3265e-01,  4.4681e-02],
        [ 1.5795e-01,  2.7841e-01,  5.0000e+00,  6.7590e-02, -2.7927e-01],
        [ 7.7088e-01,  7.9069e-01,  5.0000e+00, -8.8420e-02,  3.8760e-02],
        [ 3.3879e-01,  4.1889e-01,  5.0000e+00,  6.6469e-02,  2.2375e-01],
        [ 2.9869e-01,  3.8109e-01,  5.0000e+00,  2.5784e-02,  6.3227e-03],
        [ 7.4476e-01,  7.4959e-01,  5.0000e+00, -1.1365e-01, -1.1453e-01],
        [ 5.6607e-01,  4.0044e-01,  5.0000e+00,  9.8080e-02, -1.5553e-01]])

To provide a more convenient interface, `tomopt.muon.muon_batch.MuonBatch` is used to wrap the generated muons with methods, and property getters and setters. When instantiating a `MuonBatch`, we also need to tell it where the muons start in z. Propagation of the muons proceeds in steps of $\delta z$ in the negative z direction.

In [13]:
from tomopt.muon import MuonBatch

In [26]:
muons = MuonBatch(generate_batch(1000), init_z=1)
muons

Batch of 1000 muons

In [27]:
f'{muons.x[0]=}, {muons.y[0]=}, {muons.z[0]=}, {muons.theta[0]=}'

'muons.x[0]=tensor(0.6437), muons.y[0]=tensor(0.0102), muons.z[0]=tensor(1.), muons.theta[0]=tensor(0.0934)'

In [28]:
muons.propagate(dz=0.1)

In [29]:
f'{muons.x[0]=}, {muons.y[0]=}, {muons.z[0]=}, {muons.theta[0]=}'

'muons.x[0]=tensor(0.6346), muons.y[0]=tensor(0.0126), muons.z[0]=tensor(0.9000), muons.theta[0]=tensor(0.0934)'

Normally, though, we let the volume layers (see next section) call the `propagate` method. As the muons pass through passive material and scatter the $\theta$ values of the muons will also change (along with x & y). Detector layers will append hits with reconstructed x & y to the `MuonBatch.hits` attribute, by calling `MuonBatch.append_hits`. After traversing the entire volume, the hits can be extracted using `MuonBatch.get_hits`.

## Volume definition

### Passive volume definition
First let's set up the passive volume; a block occupying (x,y,z) space from (0,0,0)->(L,W,H), and subdivided into cubic voxels of width *size*. Each voxel can be specified as a different material with varying x0 (radiation length [m]). `tomopt.core.X0` includes a dictionary of the x0 in various materials.

The `tomopt.volume` contains methods and classes to enable the definition of the active and passive volumes. Construction of a passive volume is done layer-wise in the z-axis, and each layer should be a `tomopt.volume.layer.PassiveVolume`. These are initialised by stating the z-position of the **top** of the layer, the transverse length and width of layer, and the size of each voxel (simultaneously defines the depth of the layer and the number of voxels in the layer). The user should ensure that the length and width are both divisible by the size. The materials of the voxels in the layer are defined using a function which takes the coordinates of the layer and returns an (N,M) tensor with the X0 of material for the (NxM voxels in x,y).

Below, we'll look at how to construct a passive layer

In [4]:
import torch
from torch import Tensor

In [30]:
from tomopt.core import X0

The function below takes the layer coordinates and returns a tensor with voxel material for the layer at the specified z, In this case, it will return beryllium for all voxels, except for the layer at z=0.4, which will contain a block of lead for x_voxels > 5 and y_voxels > 5. Note that the function takes absolute coordinates in metres and must manually convert to voxel-coordinates using the size.

In [31]:
def arb_rad_length(*,z:float, lw:Tensor, size:float) -> float:
    rad_length = torch.ones(list((lw/size).long()))*X0['beryllium']
    if z >= 0.4 and z <= 0.5: rad_length[5:,5:] = X0['lead']
    return rad_length

In [32]:
from tomopt.volume import PassiveLayer

In [34]:
pl = PassiveLayer(rad_length_func=arb_rad_length, lw=Tensor([1,1]), z=0.2, size=0.1)

The `PassiveLayer` inherits from `tomopt.volume.layer.Layer`, which in turn inherits from `torch.nn.Module`, and passing a `MuonBatch` through a `Layer` involves calling its `forward` method (or calling the object, since `nn.Module.__call__` points to `forward`).

When a `MuonBatch` is passed through the `PassiveLayer`, it does so in `n` steps of $\delta z$, and at each step undergoes multiple scattering according the material traversed. Note that the `forward` method does not return the propagated `MuonBatch`, but instead updates the internal parameters of the `MuonBatch` in-place.

In [38]:
muons = MuonBatch(generate_batch(1000), init_z=0.2)
f'{muons.x[0]=}, {muons.y[0]=}, {muons.z[0]=}, {muons.theta[0]=}'

'muons.x[0]=tensor(0.9673), muons.y[0]=tensor(0.0982), muons.z[0]=tensor(0.2000), muons.theta[0]=tensor(0.1734)'

In [39]:
pl(mu=muons, n=2)

In [40]:
f'{muons.x[0]=}, {muons.y[0]=}, {muons.z[0]=}, {muons.theta[0]=}'

'muons.x[0]=tensor(0.9847), muons.y[0]=tensor(0.0981), muons.z[0]=tensor(0.1000), muons.theta[0]=tensor(0.1705)'

Note that the muons have dropped to the bottom of the passive layer, and that their $\theta$ values have also changed due to the multiple-scattering.

### Detector layer definition